In [1]:
#tsne 
from langchain_community.embeddings import HuggingFaceEmbeddings
#from langchain_chroma import Chroma
import torch
import pdb
import os
from langchain import HuggingFacePipeline
#from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
model_name  = 'Alibaba-NLP/gte-large-en-v1.5'
embedding_function = HuggingFaceEmbeddings(model_name=model_name, model_kwargs = {'trust_remote_code':True, 'device': torch.device("cpu")})
     
import pandas as pd
import numpy as np

#from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

import seaborn as sns

chroma_dir = '../scripts/PA_200c_named_db'
db = Chroma(persist_directory=chroma_dir, embedding_function=embedding_function)

# docs=db.similarity_search('what is the best chicken in Mcdonald')
# print(docs)
     

#vdb=db.get(include=["metadatas", "documents", "embeddings"])
#vdb=db.get(include=["embeddings"])
import pandas as pd
#vdb=pd.DataFrame(db)
#df = pd.DataFrame({"embeddings": vdb["embeddings"],})
vectors=db.get(include=["embeddings", "documents"])
# print(vectors)
# Convert the embeddings to a numpy array and transpose it
vec_array = np.asarray(vectors['embeddings'])
txt_array = vectors['documents']
num_test = 1800
#pdb.set_trace()
select_randidx = np.random.randint(0,len(vectors['embeddings']),num_test)
vec_array = vec_array[select_randidx, :]
print("Original array shape (transposed):", vec_array.T.shape)


datafile = '../data/200chrestReviews_PA.json'
df = pd.read_json(datafile, lines=True)
meta_df = pd.read_json('../data/meta-Pennsylvania.json', lines=True)

#search for query to get gmap id
relevant_metadata = ['Noodle', 'Thai'] 
noodle_list, thai_list = [], []

/home/mshteyn/anaconda3/envs/huggingface/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Original array shape (transposed): (1024, 1800)


In [ ]:
relevant_metadata = ['Noodle', 'Thai'] 
noodle_list, thai_list = [], []
for ii, item in enumerate(vectors["documents"]):
    find_idx = df['text'].str.contains(item[-20:-1])
    get_gmap = df[find_idx]['gmap_id']
    if len(get_gmap)>1:
        get_gmap = get_gmap.iloc[df.index[0]]        
    #entry = meta_df[meta_df['gmap'] == get_gmap]
    try:
        categories = meta_df[meta_df['gmap_id'] == get_gmap].reset_index().iloc[0].category
    except:
        pdb.set_trace()
    #string_meta = " ".join(entry.category.tolist()[0])
    string_meta = " ".join(categories)
    
    if relevant_metadata[0] in string_meta:
        noodle_list.append(ii)
    if relevant_metadata[1] in string_meta:
        thai_list.append(ii)
    
pdb.set_trace()


# Save the transposed array to a text file
# In future add request vector, convenient for plotting
text_file_name = 'array_small.text'

np.savetxt(text_file_name, vec_array.T, delimiter=',', fmt='%.18e')

# Load the array from the text file
loaded_array = np.loadtxt(text_file_name, delimiter=',')
     

## add query vector in
from sentence_transformers import SentenceTransformer
query=['For the best vegetarian plate, what should I order on the menu at Noodlehead?']

#model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
#model = model
model = SentenceTransformer(model_name, trust_remote_code=True)
query_embeddings = model.encode(query)
#print(query_embeddings.shape)
# Prepare documents for Chroma.from_documents
query_vec = query_embeddings
query_vec_array=np.asarray(query_vec)

#print(query_vec_array.T.shape)
#loaded_array = np.random.rand(384, 1)
##loaded_array = np.loadtxt('array.txt', delimiter=',')
#print(loaded_array.shape)
updated_array = np.hstack([loaded_array, query_vec_array.T])
np.savetxt(text_file_name, updated_array, delimiter=',', fmt='%.18e')
#loaded_array = np.loadtxt('array.txt', delimiter=',')
#print("Updated array shape:", loaded_array.shape)
     

> /tmp/ipykernel_18616/2827759088.py(14)<module>()
     12         pdb.set_trace()
     13     #string_meta = " ".join(entry.category.tolist()[0])
---> 14     string_meta = " ".join(categories)
     15 
     16     if relevant_metadata[0] in string_meta:



ipdb>  item


"We got food at Eat'n Park. It was my first time in a restaurant since covid 19 shut things down  My breakfast was ok gotten eggs Benedict  It would of been a lot nicer if the people at the next table didn t talk about bodily fluids   talk lower to themselves  So pretty much couldn t enjoy my breakfast cause people talking about bodily fluids etc"


ipdb>  ii


30


...

   index                                gmap_id
0     30  0x89cb11cddc8f9cdd:0xa6b8e58541d8967c


ipdb>  df[find_idx]['gmap_id'].copy().reset_index(inplace=True)


*** TypeError: Cannot reset_index inplace on a Series to create a DataFrame


ipdb>  df[find_idx]['gmap_id'].copy().reset_index(drop=True, inplace=True)
ipdb>  gmap_id = df[find_idx]['gmap_id'].copy().reset_index(drop=True, inplace=True)
ipdb>  gmap_id
ipdb>  gmap_id
ipdb>  type(gmap_id)


<class 'NoneType'>


In [ ]:

from sklearn.manifold import TSNE
import numpy as np
loaded_array = np.loadtxt(text_file_name, delimiter=',')
n_components = 2 #3D
# tsne = TSNE(n_components=n_components, random_state=1, perplexity=1)
# reduced_vectors = tsne.fit_transform(loaded_array)
     

# import matplotlib.pyplot as plt
# # Plot the reduced embeddings
# plt.scatter(reduced_vectors[:,  0], reduced_vectors[:,  1], c='red', alpha=0.2)

# # Add labels and title

# plt.show()
     

import matplotlib.pyplot as plt
# Directory to save figures
save_dir = 'tsne_figures'
# Create the directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

#parameter for tSNE list
#perplexity_col = [2,4,8,10]
perplexity_col=[50,80,100,200,300]
random_state_col=[10,20,40]


# Save figures in a loop
for i in range(len(perplexity_col)):
    for j in range(len(random_state_col)):
        tsne = TSNE(n_components=n_components, random_state=random_state_col[j], perplexity=perplexity_col[i])
        reduced_vectors = tsne.fit_transform(loaded_array.T)
        plt.figure()
        plt.scatter(reduced_vectors[:-1, 0], reduced_vectors[:-1,  1], c='purple', alpha=0.2)
        plt.scatter(reduced_vectors[-1, 0], reduced_vectors[-1,  1], marker ='*', c='red')
        filename = os.path.join(save_dir, f'figure_perplex[{perplexity_col[i]}]_randomstate[{random_state_col[j]}].png')  # Create a filename with the loop index
        plt.title("t-SNE Visualization of Text Embeddings with Question Highlighted")
        plt.xlabel("t-SNE  1")
        plt.ylabel("t-SNE  2")        
        plt.savefig(filename)  # Save the figure
        plt.close()  # Close the figure to free up memory